<h1>MassBalanceMachine XGBoost Model Training - Iceland Region Example</h1>
<p style='text-align: justify;'>
In this notebook, we will simulate the glacier surface mass balance for the Iceland region using a custom <a href='https://xgboost.readthedocs.io/en/stable/'>XGBoost</a> model. The XGBoost model is designed with a custom objective function that generates monthly predictions based on aggregated observational data. We will create an instance of <code>CustomXGBoostRegressor</code> and train it using this custom loss function on the stake data from Iceland, which we have prepared in earlier notebooks. If you haven't already, please review the <a href='https://github.com/ODINN-SciML/MassBalanceMachine/blob/main/notebooks/data_preprocessing.ipynb'>data preprocessing</a> and <a href='https://github.com/ODINN-SciML/MassBalanceMachine/blob/main/notebooks/data_preprocessing.ipynb'>data processing WGMS</a> notebooks for more details.</p>

<p style='text-align: justify;'>
The workflow includes several key steps:
</p>

<ol style="margin-left: 20px; padding-left: 0;">
    <li style="margin-bottom: 10px;">
        <p style='text-align: justify;'><strong>Data Loading and Preparation:</strong> A <code>Dataloader</code> object is created to handle the loading of data and the creation of a training and testing split. This object also manages the generation of data splits for cross-validation.</p>
    </li>
    <li style="margin-bottom: 10px;">
        <p style='text-align: justify;'><strong>Cross-Validation and Model Training:</strong> Using Scikit-learn's cross-validation techniques, we explore different hyperparameters and train the model on the prepared data splits. This approach ensures a robust evaluation and helps in selecting suitable parameters.</p>
    </li>
    <li style="margin-bottom: 10px;">
        <p style='text-align: justify;'><strong>Aggregated Predictions:</strong> After training, we will display the aggregated monthly predictions generated by the model to visualize and analyze the results.</p>
    </li>
    <li style="margin-bottom: 10px;">
        <p style='text-align: justify;'><strong>Model Evaluation:</strong> Finally, the model's performance is evaluated on the test set, providing insights into its predictive accuracy for glacier mass balance.</p>
    </li>
</ol>

In [1]:
import pandas as pd
import massbalancemachine as mbm
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

In [2]:
# Set a random seed:
SEED = 30

data = pd.read_csv('./example_data/iceland/files/iceland_monthly_dataset.csv')
print('Number of winter and annual samples:', len(data))
display(data)

Number of winter and annual samples: 447


,YEAR,POINT_LON,POINT_LAT,POINT_BALANCE,ALTITUDE_CLIMATE,ELEVATION_DIFFERENCE,RGIId,POINT_ID,ID,N_MONTHS,MONTHS,aspect,slope,t2m,tp,slhf,sshf,ssrd,fal,str
0,1995,-18.773871,64.885013,2.07,1333.923612,-116.476388,RGI60-06.00228,hn14aa,0,7,oct,1.606406,0.056246,267.885682,0.005071,-32688.346894,1.908546e+05,3.434260e+06,0.850005,-1.029337e+06
1,1995,-18.773871,64.885013,2.07,1333.923612,-116.476388,RGI60-06.00228,hn14aa,0,7,nov,1.606406,0.056246,266.376346,0.006053,301104.083653,8.280538e+05,8.424995e+05,0.849992,-1.431540e+06
2,1995,-18.773871,64.885013,2.07,1333.923612,-116.476388,RGI60-06.00228,hn14aa,0,7,dec,1.606406,0.056246,263.049011,0.005854,248241.745197,9.954409e+05,1.322171e+05,0.849992,-2.002829e+06
3,1995,-18.773871,64.885013,2.07,1333.923612,-116.476388,RGI60-06.00228,hn14aa,0,7,jan,1.606406,0.056246,261.692810,0.004156,348585.225978,1.243700e+06,4.884578e+05,0.849992,-1.792889e+06
4,1995,-18.773871,64.885013,2.07,1333.923612,-116.476388,RGI60-06.00228,hn14aa,0,7,feb,1.606406,0.056246,261.140088,0.002287,274514.643950,1.004845e+06,2.580602e+06,0.850005,-1.861757e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,2018,-18.954942,64.662707,-2.63,1067.593874,246.593874,RGI60-06.00232,blt8,54,11,jun,2.889730,0.018996,276.144439,0.003192,10994.304045,6.234279e+05,1.787212e+07,0.768536,-9.177811e+05
443,2018,-18.954942,64.662707,-2.63,1067.593874,246.593874,RGI60-06.00232,blt8,54,11,jul,2.889730,0.018996,277.553280,0.003422,-589325.605420,7.537638e+04,1.425122e+07,0.676787,-7.246837e+05
444,2018,-18.954942,64.662707,-2.63,1067.593874,246.593874,RGI60-06.00232,blt8,54,11,aug,2.889730,0.018996,276.193729,0.002495,-807105.778219,-1.563324e+05,1.332676e+07,0.677706,-1.884936e+06
445,2018,-18.954942,64.662707,-2.63,1067.593874,246.593874,RGI60-06.00232,blt8,54,11,sep,2.889730,0.018996,273.151130,0.003491,-432479.565272,1.434597e+05,9.078256e+06,0.717923,-2.152404e+06


<h2>1. Create the Train and Test Dataset and the Data Splits for Cross Validation</h2>
<p style='text-align: justify;'>
First, we create a <code>DataLoader</code> object, which generates both training and testing datasets, as well as the data splits required for cross-validation. To conserve memory, the <code>set_train_test_split</code> method returns iterators containing indices for the training and testing datasets. These indices are then used to retrieve the corresponding data for training and testing. Next, the <code>get_cv_split</code> method provides a list indicating the number of folds needed for cross-validation.
</p>


In [3]:
# Create a new DataLoader object with the monthly stake data measurements.
dataloader = mbm.DataLoader(data=data, random_seed=SEED)
# Create a training and testing iterators. The parameters are optional. The default value of test_size is 0.3.
train_itr, test_itr = dataloader.set_train_test_split(test_size=0.3,
                                                      shuffle=True)

# Get all indices of the training and testing dataset at once from the iterators. Once called, the iterators are empty.
train_indices, test_indices = list(train_itr), list(test_itr)

# Get the features and targets of the training data for the indices as defined above, that will be used during the cross validation.
df_X_train = data.iloc[train_indices]
y_train = df_X_train['POINT_BALANCE'].values

# Get test set
df_X_test = data.iloc[test_indices]
y_test = df_X_test['POINT_BALANCE'].values

# Create the cross validation splits based on the training dataset. The default value for the number of splits is 5.
splits = dataloader.get_cv_split(n_splits=5)

# Print size of train and test
print(f"Size of training set: {len(train_indices)}")
print(f"Size of test set: {len(test_indices)}")

KeyError: "['PERIOD'] not found in axis"

<h2>2. Create a CustomXGBoostRegressor Model</h2>
<p style='text-align: justify;'>
Next, we define the parameter ranges for each XGBoost parameter. In the subsequent step, we use cross-validation to explore these parameter ranges and select the combination that yields the lowest loss. Additionally, we create a <code>CustomXGBoostRegressor</code> object.
</p>

In [ ]:
# For each of the XGBoost parameter, define the grid range
parameters = {
    'max_depth': [
        3,
        4,
        5,
        6,
    ],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'n_estimators': [100, 200, 300],
    'gamma': [0, 1]
}

In [ ]:
# Create a CustomXGBoostRegressor instance
params_init = {"device": "cpu", "random_state": SEED}
custom_xgboost = mbm.models.CustomXGBoostRegressor(**params_init)

<h2>3. Train the CustomXGBoostRegressor Model</h2>

<p style='text-align: justify; margin-bottom: 5px;'>
In the following cell, we begin training our model using either <strong>GridSearchCV</strong> or <strong>RandomizedSearchCV</strong>:
</p>

<ul style="margin-left: 20px; padding-left: 0; margin-bottom: 5px;">
  <li style="margin-bottom: 10px;">
    <p style='text-align: justify;'><strong>GridSearchCV</strong> performs an exhaustive search across all possible parameter combinations to find the best set for optimal performance using cross-validation. While this method is thorough, it is often time-consuming and computationally expensive.</p>
  </li>
  <li style="margin-bottom: 0px;">
    <p style='text-align: justify;'><strong>RandomizedSearchCV</strong>, on the other hand, samples a fixed number of parameter combinations from the distribution, making it more efficient in terms of time and computational resources, especially with larger hyperparameter spaces. However, this approach may miss some of the best parameter combinations that aren't selected in the random sampling.</p>
  </li>
</ul>

<p style='text-align: justify;'>
You can choose either of the two training methods. Both methods will use all CPU cores by default. If you want to adjust the number of cores used, you can change the <code>num_jobs</code> parameter.
</p>


In [ ]:
# GridSearch
# custom_xgboost.gridsearch(parameters=parameters, splits=splits, features=df_X_train, targets=y_train, num_jobs=-1)

# RandomisedSearch, with n_iter the number of parameter settings that are sampled. Trade-off between goodness of the solution
# versus runtime.
custom_xgboost.randomsearch(
    parameters=parameters,
    n_iter=20,
    splits=splits,
    features=df_X_train,
    targets=y_train,
    num_jobs=-1,
    random_seed=SEED,
)
best_params = params = custom_xgboost.param_search.best_params_
best_estimator = custom_xgboost.param_search.best_estimator_
print("Best parameters:\n", best_params)
print("Best score:\n", custom_xgboost.param_search.best_score_)

<h4>3.1 Save the Trained Model</h4>

In [ ]:
# custom_xgboost.save_model('model.pkl')

In [ ]:
# mbm.models.CustomXGBoostRegressor.load_model('model.pkl')

<h3>3.1 Show the Predictions per Fold</h3>

In [ ]:
def predVSTruth(grouped_ids, mae, rmse, title):
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    legend_xgb = "\n".join(
        (r"$\mathrm{MAE_{xgb}}=%.3f, \mathrm{RMSE_{xgb}}=%.3f$ " % (
            mae,
            rmse,
        ), ))

    marker_xgb = 'o'
    sns.scatterplot(grouped_ids,
                    x="target",
                    y="pred",
                    ax=ax,
                    alpha=0.5,
                    marker=marker_xgb)

    ax.set_ylabel('Predicted PMB [m w.e.]', fontsize=20)
    ax.set_xlabel('Observed PMB [m w.e.]', fontsize=20)

    ax.text(0.03,
            0.98,
            legend_xgb,
            transform=ax.transAxes,
            verticalalignment="top",
            fontsize=20)
    ax.legend([], [], frameon=False)
    # diagonal line
    pt = (0, 0)
    ax.axline(pt, slope=1, color="grey", linestyle="-", linewidth=0.2)
    ax.axvline(0, color="grey", linestyle="-", linewidth=0.2)
    ax.axhline(0, color="grey", linestyle="-", linewidth=0.2)
    ax.grid()
    ax.set_title(title, fontsize=20)
    plt.tight_layout()

In [ ]:
# Set to CPU for predictions:
xgb = best_estimator.set_params(device='cpu')

meta_data_columns = ["RGIId", "POINT_ID", "ID", "N_MONTHS", "MONTHS"]

# Make predictions on test
features_test, metadata_test = xgb._create_features_metadata(
    df_X_test, meta_data_columns)
y_pred = xgb.predict(features_test)

# Make predictions aggr to meas ID:
y_pred_agg = xgb.aggrPredict(metadata_test, meta_data_columns, features_test)

# Calculate scores
score = xgb.score(df_X_test, y_test)  # negative
mse, rmse, mae, pearson = xgb.evalMetrics(metadata_test, y_pred, y_test)

# Aggregate predictions to annual or winter:
df_pred = df_X_test.copy()
df_pred['target'] = y_test
grouped_ids = df_pred.groupby('ID').agg({'target': 'mean'})
grouped_ids['pred'] = y_pred_agg

predVSTruth(grouped_ids, mae, rmse, title='XGBoost on test')